In [35]:
import os 
import time as t
import csv
import numpy
import pandas as pd
import random 
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.svm import SVC

In [88]:
  from Preprocess import Preprocessing

In [78]:
def Reading_data(filename1,filename2):
 
    # reading given csv file 
    # and creating dataframe
    websites = pd.read_csv(filename1 ,header = None)

    # adding column headings
    websites.columns = ['Sentiment', 'labels']

    # store dataframe into csv file
    websites.to_csv(filename2, index = None)

    df = pd.read_csv(filename2)
    #getting rid of null values
    df = df.dropna()
    #Taking a 30% representative sample
    np.random.seed(34)
    df1 = df.sample(frac = 1)
    #Adding the sentiments column
    df1['labels'] = df1.labels.apply(lambda x: 0 if x=='adverbs' else 1)
    
    X = df1['Sentiment']
    y = df1['labels']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=34)
    X_test.to_csv('adjective_adverb_corpus/X_test.csv', index = None)
    y_test.to_csv('adjective_adverb_corpus/y_test.csv', index = None)




In [89]:
from sklearn.linear_model import SGDClassifier
def Svm_model(X_train,X_test):
    vectorizer = TfidfVectorizer()
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    from sklearn import svm
    svcl = svm.SVC(kernel="linear")

    svcl.fit(X_train_vec, y_train)
    from sklearn.metrics import classification_report
    # Perform classification with SVM, kernel=linear
   


In [90]:

def Svm_model_secor(X_test,y_test):
    X_test_vec = vectorizer.transform(X_test)
    svcl_score = svcl.score(X_test_vec, y_test)
    print("Results for Support Vector Machine with tfidf")
    print(svcl_score)
    y_pred_sv = svcl.predict(X_test_vec)
    #Confusion matrix
    cm_sv = confusion_matrix(y_test, y_pred_sv)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_sv).ravel()
    print(tn, fp, fn, tp)
    tpr_sv = round(tp/(tp + fn), 4)
    tnr_sv = round(tn/(tn+fp), 4)
    print(tpr_sv, tnr_sv)
  

In [91]:
def After_Spilt_data():
    data_x= pd.read_csv("adjective_adverb_corpus/X_test.csv")
    Xtest = data_x['Sentiment']
    data_y = pd.read_csv("adjective_adverb_corpus/y_test.csv")
    ytest= data_y['labels']
    return Xtest,ytest

After split data test the model

In [92]:
filename1='adjective_adverb_corpus/Train.txt'
filename2='adjective_adverb_corpus/Train.csv'
Reading_data(filename1,filename2)
Svm_model(X_train,X_test)


Passing x_test and Y_test data for predict the accuracy of the modeel

In [93]:
Svm_model_secor(X_test,y_test)

Results for Support Vector Machine with tfidf
0.5014765100671141
1787 44 1813 81
0.0428 0.976


In [94]:
file_name_3='adjective_adverb_corpus/Train.txt'
file_name_1='adjective_adverb_corpus/adjectives.txt'
file_name_2='adjective_adverb_corpus/adverbs.txt'
Preprocessing(file_name_1 ,file_name_2,file_name_3)
Reading_data(filename1,filename2)

In [95]:
After_Spilt_data()
Svm_model_secor(Xtest,ytest)

Results for Support Vector Machine with tfidf
0.5014765100671141
1787 44 1813 81
0.0428 0.976


In [96]:

kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

np.random.seed(1)

pipeline_svm = make_pipeline(vectorizer, 
                            SVC(probability=True, kernel="linear", class_weight="balanced"))

grid_svm = GridSearchCV(pipeline_svm,
                    param_grid = {'svc__C': [0.01, 0.1, 1]}, 
                    cv = kfolds,
                    scoring="roc_auc",
                    verbose=1,   
                    n_jobs=-1) 

grid_svm.fit(X_train, y_train)
grid_svm.score(X_test, y_test)


Fitting 5 folds for each of 3 candidates, totalling 15 fits


0.5133505329140227

In [97]:
grid_svm.best_params_

{'svc__C': 1}

In [98]:
grid_svm.best_score_

0.514479419004559

In [99]:
review = """accommodatingly"""
review_vector = vectorizer.transform([review]) # vectorizing
print(svcl.predict(review_vector))

[0]
